# Container Object

- 컨테이너는 `__contains__`메소드를 구현한 객체이다.
- `__contains__`는 일반적으로 `Boolean`값을 반환한다.
- 이러한 메소드는 파이썬에서 `in` 키워드가 발견될 때, 사용된다.
- 해당 메소드를 잘 사용하면 **가독성**이 좋아진다.

<br/>

```python
element in container
```

<br/>

위의 코드를 파이썬에서는 아래와 같이 해석한다.
```python
container.__contains__(element)
```

<br/>

위에서 `__contains__`메소드를 잘 사용하면, **가독성**이 좋아진다고 언급했는데, 아래의 코드를 보자

<br/>

```python
def mark_coordinate(grid, coord):
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED
```

- 위의 코드에서 `if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:`줄은 직관적으로 이해하기 어렵다.
- 이를 grid라는 class로 판단하며, 실제 처리 방식을 더 작은 객체에게 위임하면 가독성이 좋아지게된다.
> 책에서는 **위임을 통해 응집력도 높아진다.** 라고 언급하는데, 해당 의미는 무엇일까?...  
    
이를 `__contains__`로 리팩토링해보자  
```python
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def __contains__(self, coord):
        x, y = coord
        return 0 <= coord.x < grid.width and 0 <= coord.y < grid.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)
        
    def __contains__(self, coord):
        return coord in self.limits
```

위와 같이 작성하면, `mark_coordinate` 함수는 아래와 같이 간결해진다.
```python
def mark_coordinate(grid, coord):
    if coord in grid:
        grid[coord] = MARKED
```

# Get attribute

- `__getattr__` 매직 메소드를 이용하면, 객체에서 속성을 얻는 방법을 제어할 수 있다.
- 객체에서 속성을 얻는 방법이라고 하면 아래와 같은 클래스가 있을 때, 해당 객체의 멤버 변수나, 멤버 함수를 가져오는 것들을 이야기한다. 아래의 코드에서는 `person.name`, `person.number`와 같이 **[myobject].[myattribute]** 형태를 말한다. 
- 만약 객체에 해당 속성이 없다면, 속성(myattribute)의 이름을 파라미터로 전달하여 `__getattr__`이라는 추가 메소드를 호출한다.

```python
class Person:
    def __init__(self, name, number):
        self.name = name
        self.number

person = Person("Martin", "010-xxxx-xxxx")
name = person.name
number = person.number
```

<br/>

```python
class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
        
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음.")
```

위에서 정의한 `DynamicAttributes`는 아래와 같이 사용할 수 있다.  
    
```bash
> dyn = DynamicAttributes("value")
> dyn.attribute
'value'

> dyn.fallback_test
'[fallback resolved] test'

> dyn.__dict__["fallback_new"] = "new value"
> dyn.fallback_new
'new value'

> getattr(dyn, "something", "default")
'default'
```

<br/>

해당 코드는 아래의 코드와 같은 의미다. 하지만 단순히 `__getattr__`메소드가 실행되지 않았기 때문에, `fallback_new`라는 속성이 생긴다.

```python
> dyn.__dict__["fallback_new"] = "new value"
> dyn.fallback_new
'new value'
```

<br/>


```python
> dyn.fallback_new = "new value"
```

## Should raise `AttributeError` in `__getattr__!!!`

- 아래와 같이 `__getattr__` 메소드는 `AttributeError`를 발생시켜야한다.

```python
def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"

        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음.")
```

In [18]:
class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
        
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return "[fallback resolved] {}".format(name)
        
        raise AttributeError("{}에는 {} 속성이 없음.".format(self.__class__.__name__, attr))
        
dyn = DynamicAttributes("value")
print(dyn.attribute)
print(dyn.fallback_test)
dyn.__dict__["fallback_new"] = "new value"
print(dyn.fallback_new)
print(getattr(dyn, "something", "default"))

value
[fallback resolved] test
new value
default


## String startwith() method
- **startwith** 함수는 파라미터로 받은 String이 해당 문자열로 시작하는지를 반환한다.
- `beg`, `end`는 특정 인덱스 안에서 해당 String이 해당 문자열로 시작하는지 확인할 수 있다.  

```python
str.startwith(str, beg=0, end=len(string))
```
### Parameters

- **str** − This is the string to be checked.

- **beg** − This is the optional parameter to set start index of the matching boundary.

- **end** − This is the optional parameter to end start index of the matching boundary.


In [8]:
string = "this is string example....wow!!!";
print(string.startswith( 'this' ))
print(string.startswith( 'is', 2, 4 ))
print(string.startswith( 'this', 2, 4 ))
print(string.startswith( 'string', 2, 4 ))
print(string.startswith( 'string', 8, 15 ))

True
True
False
False
True


# Callable object

- 객체가 함수처럼 작동되면 매우 편하다.
- 이는 데코레이터를 이용해 만들 수도 있지만, 매직 메소드를 사용해서 할 수도 있다.
- `__call__` 매직 메소드를 이용해서 객체를 일반 함수처럼 호출한다.
- 이렇게 사용하는 이유는 함수에는 상태를 저장할 수 없지만, 객체에서는 상태를 저장할 수 있기 때문이다.
> 즉, 상태를 저장할 수 있는 함수로써 사용하고 싶은 것이다.
- 파이썬은 `object(*args, **kwargs)`와 같은 구문을 `object.__call__(*args, **kwargs)`로 변환한다.

In [19]:
from collections import defaultdict

class CallCount:
    
    def __init__(self):
        self._counts = defaultdict(int)
        
    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]
    
cc = CallCount()
print(cc(1))
print(cc(2))
print(cc(1))
print(cc(1))
print(cc("something"))

1
1
2
3
1


# Should be becareful in Python

- 방어코드를 작성하지 않으면, 오랜 시간 디버깅하는데 고생할만한 이슈들을 살펴봄

## 변경 가능한(mutable) 파라미터의 기본 값

- 변경 가능한 객체를 함수의 기본 인자로 사용하면 안된다.
```python
def wrong_user_display(user_metadata: dict = {"name": "John", "age": 30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    
    return "{} ({})".format(name, age)
```

- `user_metadata`는 `dict`타입으로 변경이 가능하다.
- 함수 내에서 가변객체를 수정한다.
- `user_metadata`의 기본인자도 문제다.
    - default값을 이용해 함수를 호출하면, 기본데이터로 사용할 dict을 한 번만 생성한다.
    - 함수 본체에서 해당 dict을 사용하고 수정한다.
    - 두번째 호출에서 다른 파라미터를 사용하면, 기본데이터 대신 받은 파라미터를 사용한다.
    - 따라서 세번째 호출에서 다른 파라미터를 사용하지 않고, 기본 값을 이용해 호출하면 실패한다.
    - 첫번째 호출 시, key를 지웠기 때문이다.
    
> - 위 방법에 대한 해결책은 초기값을 `None`으로 사용한다.
> - 함수 본문에서 기본 값을 할당해서 쓴다.

In [20]:
def wrong_user_display(user_metadata: dict = {"name": "John", "age": 30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    
    return "{} ({})".format(name, age)

print(wrong_user_display())
print(wrong_user_display({"name" : "Jane", "age" : 25}))
print(wrong_user_display())

John (30)
Jane (25)


KeyError: 'name'

## Built-in 타입 확장

- python에 내장되어있는 `list`, `dict`, `str`과 같은 내장타입을 클래스로 확장(상속)할때는 **collection** 모듈을 사용한다.
- 그 이유는 Cython, PyPy와 호환성을 위함이다.